### Задача исследования

Необходимо проанализировать базу данных и выяснить:

* сколько книг вышло после 1 января 2000 года;
* количество обзоров и среднюю оценку (для каждой книги);
* издательство, которое выпустило наибольшее число книг толще 50 страниц;
* автора с самой высокой средней оценкой книг;
* среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Цель исследования

Ответы на вопросы выше нацелены на формирование ценностного предложения для нового продукта - сервиса для чтения книг по подписке. Ценностное предложение поможет определить ряд преимуществ, которые потребители получат при покупке продукта.

### Датасет

* books - содержит данные о книгах;
* authors - содержит данные об авторах;
* publishers - содержит данные об издательствах;
* ratings - содержит данные о пользовательских оценках книг;
* reviews - содержит данные о пользовательских обзорах на книги.

In [1]:
# импортируем библиотеки

import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры

db_config = {'user': '', # имя пользователя
    'pwd': '', # пароль
    'host': '',
    'port': , # порт подключения
    'db': ''} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'])
 
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
#выведем таблицу books

query = '''SELECT *
           FROM books
        '''

In [4]:
books = pd.io.sql.read_sql(query, con = engine) 

books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
#выведем таблицу authors

query = '''SELECT *
           FROM authors
           LIMIT 5
        '''

In [6]:
authors = pd.io.sql.read_sql(query, con = engine) 

authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
#выведем таблицу publishers

query = '''SELECT *
           FROM publishers
           LIMIT 5
        '''

In [8]:
publishers = pd.io.sql.read_sql(query, con = engine) 

publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
#выведем таблицу ratings

query = '''SELECT *
           FROM ratings
           LIMIT 5
        '''

In [10]:
ratings = pd.io.sql.read_sql(query, con = engine) 

ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
#выведем таблицу reviews

query = '''SELECT *
           FROM reviews
           LIMIT 5
        '''

In [12]:
reviews = pd.io.sql.read_sql(query, con = engine) 

reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Подгрузили датасеты, видим, что связь с базой есть и все таблицы считались.

#### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [13]:
query = '''SELECT COUNT(title)
           FROM books
           WHERE publication_date > '2000-01-01'           
        '''

In [14]:
pd.io.sql.read_sql(query, con = engine) 

,count
0,819


**С 1 января 2000 года вышло 819 книг.**

#### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [28]:
query = '''SELECT avg_rating.book_id,
                  avg_rating.title,
                  COUNT(r.review_id),
                  avg_rating.avg
           FROM (
                 SELECT b.book_id,
                 b.title,
                 AVG(r.rating)       
                 FROM ratings AS r
                 LEFT JOIN books AS b ON r.book_id = b.book_id
                 GROUP BY b.book_id
                 ) AS avg_rating
           LEFT JOIN reviews AS r ON r.book_id = avg_rating.book_id
           GROUP BY 1,
                    2,
                    4     
        '''

In [29]:
pd.io.sql.read_sql(query, con = engine) 

,book_id,title,count,avg
0,200,Dreamcatcher,3,2.800000
1,593,Sputnik Sweetheart,3,3.666667
2,735,The God Delusion,4,4.000000
3,570,Silent Spring,2,3.000000
4,604,Surely You're Joking Mr. Feynman!: Adventures...,3,4.250000
...,...,...,...,...
995,70,American Gods (American Gods #1),5,4.000000
996,488,Of Mice and Men,5,3.622951
997,886,The Tombs of Atuan (Earthsea Cycle #2),3,4.666667
998,881,The Tenth Circle,4,3.500000


**Построили таблицу с количеством обзоров и средней оценкой каждой книги. Наибольшее количество обзоров получили:**

* Twilight (Twilight #1) - 7 обзоров, при этом средняя оценка книги - 3,66;
* The Da Vinci Code (Robert Langdon #2) - 6 обзоров, при этом средняя оценка книги - 3,83;
* The Road - 6 обзоров, при этом средняя оценка книги - 3,77;

**Наивысшие оценки имеют книги с маленьким количеством оценок - всего 2 или 3 оценки на книгу, например Pop Goes the Weasel (Alex Cross #5 или The Ghost Map).**

**При этом самые низкие рейтинги получили также книги с маленьким количеством оценок - всего 2 или 3 на книгу, например, Harvesting the Heart, которая имеет средний рейтинг 1,5 при 2 оценках.**

**Всего было поставлего 2 793 оценок, средняя оценка всех книг - 3,92**

In [17]:
reviews['review_id'].count()

5

#### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [18]:
query = '''SELECT p.publisher  
           FROM books AS b
           JOIN publishers AS p ON b.publisher_id = p.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher_id
           ORDER BY COUNT(b.book_id) DESC
           LIMIT 1
        '''

In [19]:
pd.io.sql.read_sql(query, con = engine) 

,publisher
0,Penguin Books


**Наибольшее количество книг выпустило издательство Penguin Books - 42 книги более 50 страниц каждая**

**А меньше всего - Dial Books - только 1 книга**

#### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [20]:
query = '''SELECT a.author,
                  AVG(r.rating)
           FROM books AS b
           LEFT JOIN ratings AS r ON  b.book_id = r.book_id
           LEFT JOIN authors AS a ON  b.author_id = a.author_id
           WHERE b.book_id IN (
                               SELECT book_id
                               FROM ratings
                               GROUP BY book_id
                               HAVING COUNT(rating_id) >= 50
                               )
           GROUP BY a.author
           ORDER BY avg(r.rating) DESC
           LIMIT 1
        '''

In [21]:
pd.io.sql.read_sql(query, con = engine) 

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


**J.K. Rowling и Mary GrandPré создают книги с самым высоким средним рейтингом - 4.29**

**А вот John Steinbeck не такой успешный - его книги в среднем имеют рейтинг 3.62**

#### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [22]:
query = '''SELECT AVG(cnt)
           FROM (
                 SELECT COUNT(review_id)
                 FROM reviews
                 WHERE username in (
                              SELECT username
                              FROM ratings
                              GROUP BY username
                              HAVING COUNT(rating_id) > 50
                              )
                 GROUP BY username) AS cnt_rew(cnt) 
        '''

In [23]:
pd.io.sql.read_sql(query, con = engine) 

,avg
0,24.333333


**В среднем пользователи, которые поставили более 50 оценок, оставляют примерно 24 обзора, то есть примерно половину от всех оценок.**

**Эти пользователи с более чем 50 оценок - jennifermiller (53), martinadam(56), paul88(56), richard89(55), sfitzgerald(55) и xdavis(51)**

### Выводы



Приобретенный сервис имеет следующие характеристики:
    
С 1 января 2000 года вышло 819 книг.

Построили таблицу с количеством обзоров и средней оценкой каждой книги. Наибольшее количество обзоров получили:

* Twilight (Twilight #1) - 7 обзоров, при этом средняя оценка книги - 3,66;
* The Da Vinci Code (Robert Langdon #2) - 6 обзоров, при этом средняя оценка книги - 3,83;
* The Road - 6 обзоров, при этом средняя оценка книги - 3,77;

Наивысшие оценки имеют книги с маленьким количеством оценок - всего 2 или 3 оценки на книгу, например Pop Goes the Weasel (Alex Cross #5 или The Ghost Map).

При этом самые низкие рейтинги получили также книги с маленьким количеством оценок - всего 2 или 3 на книгу, например, Harvesting the Heart, которая имеет средний рейтинг 1,5 при 2 оценках.

**Всего было поставлего 2 793 оценок, средняя оценка всех книг - 3,92**

Наибольшее количество книг выпустило издательство Penguin Books - 42 книги более 50 страниц каждая. А меньше всего - Dial Books - только 1 книга.

J.K. Rowling и Mary GrandPré создают книги с самым высоким средним рейтингом - 4.29. А вот John Steinbeck не такой успешный - его книги в среднем имеют рейтинг 3.62.

В среднем пользователи, которые поставили более 50 оценок, оставляют примерно 24 обзора, то есть примерно половину от всех оценок. Эти пользователи с более чем 50 оценок - jennifermiller (53), martinadam(56), paul88(56), richard89(55), sfitzgerald(55) и xdavis(51).

#### Цель исследования

В целом ценностное предложение можно сформулировать следующим образом: с помощью подписочной системы пользователь приобретает возможность прочитать популярные (и любимые - судя по рейтигам) книги, оставить оценку книге и отзыв о прочтении. Вы также сможете прочитать отзывы других пользователей и посмотреть оценки книг.